In [1]:
import pandas as pd
import numpy as np
from scipy import stats

### Коэффициенты корреляции


In [2]:
data = pd.read_csv('illiteracy.txt', sep='\t')

In [3]:
round(data[['Illit', 'Births']].corr().min().min(), 4)

0.7687

In [4]:
round(scipy.stats.spearmanr(data[['Illit', 'Births']].values).correlation, 4)

NameError: name 'scipy' is not defined

### Корреляционный анализ

In [5]:
data = pd.read_csv('water.txt', sep='\t')

In [6]:
round(data.corr().iloc[0, 1], 4)

-0.6548

In [7]:
data.columns

Index(['location', 'town', 'mortality', 'hardness'], dtype='object')

In [8]:
round(scipy.stats.spearmanr(data[['mortality', 'hardness']].values).correlation, 4)

NameError: name 'scipy' is not defined

---

In [9]:
data

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18
...,...,...,...,...
56,South,Walsall,1527,60
57,South,West Bromwich,1627,53
58,South,West Ham,1486,122
59,South,Wolverhampton,1485,81


In [10]:
df_1 = data[data['location'] == 'South']
df_2 = data[data['location'] == 'North']

In [11]:
round(df_1.corr().iloc[0, 1], 4)

-0.6022

In [12]:
round(df_2.corr().iloc[0, 1], 4)

-0.3686

---

In [13]:
stats.chi2_contingency([[239, 203], [515, 718]])

(19.40753078854304,
 1.0558987006638725e-05,
 1,
 array([[198.96597015, 243.03402985],
        [555.03402985, 677.96597015]]))

---

In [14]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [15]:
data_m = np.hstack([np.ones(239), np.zeros(515)])
data_w = np.hstack([np.ones(203), np.zeros(718)])

In [16]:
round(proportions_diff_confint_ind(data_w, data_m)[0], 4)

NameError: name 'scipy' is not defined

---

In [17]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [18]:
z = proportions_diff_z_stat_ind(data_w, data_m)
proportions_diff_z_test(z)

NameError: name 'scipy' is not defined

---

In [19]:
round(stats.chi2_contingency([[197, 111, 33], [382, 685, 331], [110, 342, 333]])[0], 4)

293.6831

In [20]:
stats.chi2_contingency([[197, 111, 33], [382, 685, 331], [110, 342, 333]])[1]

2.4964299580093467e-62

In [21]:
stats.chisquare([[197, 111, 33], [382, 685, 331], [110, 342, 333]])

Power_divergenceResult(statistic=array([168.03773585, 439.7943761 , 256.53945481]), pvalue=array([3.24391365e-37, 3.16129223e-96, 1.96410723e-56]))

---

In [22]:
def cramers_corrected_stat(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher, 
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))

In [23]:
round(cramers_corrected_stat(np.array([[197, 111, 33], [382, 685, 331], [110, 342, 333]])), 4)

0.2396

### Множественная проверка гипотез

In [28]:
from itertools import combinations
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.sandbox.stats.multicomp import multipletests 

In [25]:
data = pd.read_csv('AUCs.txt', sep='\t')

In [49]:
cols = ['C4.5', 'C4.5+m', 'C4.5+cf', 'C4.5+m+cf']
res_list = []
for i, j in combinations(cols, 2):
    st = stats.wilcoxon(data[i] - data[j], correction=True)
    res_list.append([i, j, st[0], st[1]])
    print(i, j, st)
res_df = pd.DataFrame(res_list, columns=['model1', 'model2', 'corr', 'p'])

C4.5 C4.5+m WilcoxonResult(statistic=6.5, pvalue=0.012030358470209076)
C4.5 C4.5+cf WilcoxonResult(statistic=43.0, pvalue=0.888806954437247)
C4.5 C4.5+m+cf WilcoxonResult(statistic=11.0, pvalue=0.017496137183767736)
C4.5+m C4.5+cf WilcoxonResult(statistic=17.0, pvalue=0.05030093442929343)
C4.5+m C4.5+m+cf WilcoxonResult(statistic=22.0, pvalue=0.35029075975557733)
C4.5+cf C4.5+m+cf WilcoxonResult(statistic=10.0, pvalue=0.025369859822053694)


In [50]:
res_df

,model1,model2,corr,p
0,C4.5,C4.5+m,6.5,0.012030
1,C4.5,C4.5+cf,43.0,0.888807
2,C4.5,C4.5+m+cf,11.0,0.017496
3,C4.5+m,C4.5+cf,17.0,0.050301
4,C4.5+m,C4.5+m+cf,22.0,0.350291
5,C4.5+cf,C4.5+m+cf,10.0,0.025370


In [51]:
reject, p_corrected, a1, a2 = multipletests(res_df['p'], 
                                            alpha = 0.05, 
                                            method = 'holm')
res_df['p_corrected'] = p_corrected
res_df['reject'] = reject

In [52]:
res_df

,model1,model2,corr,p,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.012030,0.072182,False
1,C4.5,C4.5+cf,43.0,0.888807,0.888807,False
2,C4.5,C4.5+m+cf,11.0,0.017496,0.087481,False
3,C4.5+m,C4.5+cf,17.0,0.050301,0.150903,False
4,C4.5+m,C4.5+m+cf,22.0,0.350291,0.700582,False
5,C4.5+cf,C4.5+m+cf,10.0,0.025370,0.101479,False


In [53]:
reject, p_corrected, a1, a2 = multipletests(res_df.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 
res_df['p_corrected'] = p_corrected
res_df['reject'] = reject
res_df

,model1,model2,corr,p,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.012030,0.050740,False
1,C4.5,C4.5+cf,43.0,0.888807,0.888807,False
2,C4.5,C4.5+m+cf,11.0,0.017496,0.050740,False
3,C4.5+m,C4.5+cf,17.0,0.050301,0.075451,False
4,C4.5+m,C4.5+m+cf,22.0,0.350291,0.420349,False
5,C4.5+cf,C4.5+m+cf,10.0,0.025370,0.050740,False


### Практика построения регрессии

In [161]:
data = pd.read_csv('botswana.tsv', sep='\t')
data['nevermarr'] = data['agefm'].isna().astype('int')
data = data.drop('evermarr', axis=1)
data['agefm'] = data['agefm'].fillna(0)

In [162]:
data.loc[data['nevermarr'] == 1, 'heduc'] = -1

In [163]:
data['heduc'].isna().sum()

123

---

In [164]:
data['idlnchld_noans'] = data['idlnchld'].isna().astype('int')
data['idlnchld'] = data['idlnchld'].fillna(-1)
data['heduc_noans'] = data['heduc'].isna().astype('int')
data['heduc'] = data['heduc'].fillna(-2)
data['usemeth_noans'] = data['usemeth'].isna().astype('int')
data['usemeth'] = data['usemeth'].fillna(-1)

In [165]:
df = data.dropna()
df.shape[0] * df.shape[1]

78264

---

In [167]:
import statsmodels.formula.api as smf

In [168]:
df.columns

Index(['ceb', 'age', 'educ', 'religion', 'idlnchld', 'knowmeth', 'usemeth',
       'agefm', 'heduc', 'urban', 'electric', 'radio', 'tv', 'bicycle',
       'nevermarr', 'idlnchld_noans', 'heduc_noans', 'usemeth_noans'],
      dtype='object')

In [169]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted = m1.fit()
print(fitted.summary())


                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Thu, 19 Mar 2020   Prob (F-statistic):               0.00
Time:                        17:39:21   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [170]:
import statsmodels.stats.api as sms

print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [171]:
m4 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted = m4.fit(cov_type='HC1')
print(fitted.summary())


                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Thu, 19 Mar 2020   Prob (F-statistic):               0.00
Time:                        17:40:20   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [176]:
m5 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted = m5.fit(cov_type='HC1')
print(fitted.summary())
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.7
Date:                Thu, 19 Mar 2020   Prob (F-statistic):               0.00
Time:                        17:41:34   Log-Likelihood:                -7735.7
No. Observations:                4349   AIC:                         1.550e+04
Df Residuals:                    4334   BIC:                         1.560e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [177]:
print("F=%f, p=%.4f, k1=%f" % m4.fit().compare_f_test(m5.fit()))

F=0.767320, p=0.5956, k1=6.000000


In [178]:
m6 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans', 
             data=data)
fitted = m6.fit(cov_type='HC1')
print(fitted.summary())
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     396.6
Date:                Thu, 19 Mar 2020   Prob (F-statistic):               0.00
Time:                        17:42:13   Log-Likelihood:                -7827.0
No. Observations:                4349   AIC:                         1.568e+04
Df Residuals:                    4336   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1932      0.262     -4.

In [179]:
m5.fit().compare_f_test(m6.fit())

(92.91524538041602, 3.0801200123461483e-40, 2.0)